In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

output_file_path = 'new_plan.xlsx'

# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = r'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

env_file_path = 'D:/Projects/.env'
load_dotenv(env_file_path)
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_ASKGLOBAL")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)
invoice_numbers_df = pd.read_excel(r'D:\Projects\BOT\invoiceNumber.xlsx')

# Extract invoice IDs as a list of strings
invoice_numbers = invoice_numbers_df['InvoiceNumber'].astype(str).tolist()

sql_query = f"""
SELECT O.Name AS Base,
       D.Name AS DocKind,
       CAST(i.Number AS varchar) AS InvoiceNumber,
       G.GoodId AS Goodid,
       G.Name AS GoodName,
       M.Name AS Producer,
       C.Inn AS Inn,
       C.FindName AS ClientName,
       PIM.Name AS InvoiceManager,
       PCM.Name AS ClientManager,
       I.PaymentTerm,
       il.pBasePrice AS BasePrice,
       il.Price AS SellingPrice,
       il.Kolich AS Quantity,
       I.DataEntered AS DataEntered,
       il.pBasePrice * il.Kolich AS BaseAmount,
       il.pSumma AS TotalAmount
FROM INVOICELN il
JOIN INVOICE i ON il.InvoiceId = i.InvoiceId
JOIN PERSONAL PIM ON i.PersonalId = PIM.PersonalId
JOIN DOCKIND D ON i.DocKindId = D.DocKindId
JOIN CLIENT C ON i.ClientId = C.ClientId
JOIN INCOMELN incl ON il.IncomeLnId = incl.IncomeLnId
JOIN Good G ON incl.GoodId = G.GoodId
JOIN Producer M ON M.ProducerId = G.ProducerId
JOIN PERSONAL PCM ON C.PersonalId = PCM.PersonalId
JOIN OFFICE O ON C.OfficeId = O.OfficeId
WHERE YEAR(I.DataEntered) >= 2024 
AND i.Number = ?
ORDER BY i.DataEntered DESC;
"""

result_dfs = []

# Iterate through each invoice ID, execute the query, and append the result to the list
for invoice_id in invoice_numbers:
    df = pd.read_sql_query(sql_query, engine, params=(invoice_id,))
    result_dfs.append(df)

# Combine all dataframes into a single dataframe
df = pd.concat(result_dfs, ignore_index=True)

# Filter out empty or all-NA columns
df = df.dropna(axis=1, how='all')

df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientManager', right_on='ClientMan', how='left')

df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')


In [ ]:
inn_list = [305088417, 305889069, 203043635, 203292026, 304600967, 206650306, 207165228, 309501746, 201282467,
            303451283, 305751344, 206997166, 302203228, 301159568, 207100940, 310046238, 302864363, 302864363, ]
sql_query = f"""
SELECT O.Name AS Base,
       D.Name AS DocKind,
       CAST(i.Number AS varchar) AS InvoiceNumber,
       G.GoodId AS Goodid,
       G.Name AS GoodName,
       M.Name AS Producer,
       C.Inn AS Inn,
       C.FindName AS ClientName,
       PIM.Name AS InvoiceManager,
       PCM.Name AS ClientManager,
       I.PaymentTerm,
       il.pBasePrice AS BasePrice,
       il.Price AS SellingPrice,
       il.Kolich AS Quantity,
       I.DataEntered AS DataEntered,
       il.pBasePrice * il.Kolich AS BaseAmount,
       il.pSumma AS TotalAmount
FROM INVOICELN il
JOIN INVOICE i ON il.InvoiceId = i.InvoiceId
JOIN PERSONAL PIM ON i.PersonalId = PIM.PersonalId
JOIN DOCKIND D ON i.DocKindId = D.DocKindId
JOIN CLIENT C ON i.ClientId = C.ClientId
JOIN INCOMELN incl ON il.IncomeLnId = incl.IncomeLnId
JOIN Good G ON incl.GoodId = G.GoodId
JOIN Producer M ON M.ProducerId = G.ProducerId
JOIN PERSONAL PCM ON C.PersonalId = PCM.PersonalId
JOIN OFFICE O ON C.OfficeId = O.OfficeId
WHERE YEAR(I.DataEntered) = 2023 and MONTH(I.DataEntered) = 9 and 
C.Inn =?
ORDER BY i.DataEntered DESC;
"""
result_dfs = []
# Iterate through each invoice ID, execute the query, and append the result to the list
for inn in inn_list:
    df = pd.read_sql_query(sql_query, engine, params=(str(inn),))
    result_dfs.append(df)

# Combine all dataframes into a single dataframe
df = pd.concat(result_dfs, ignore_index=True)

# Filter out empty or all-NA columns
df = df.dropna(axis=1, how='all')

df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientManager', right_on='ClientMan', how='left')

df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')

In [ ]:
df.to_excel('salom.xlsx')

In [ ]:
import os
from datetime import datetime, timedelta  # For working with dates

import pandas as pd  # For working with DataFrames
from dotenv import load_dotenv
from sqlalchemy import create_engine  # For creating a connection engine


def hourly_generator(login, password):
    ##################### LOADING IMPORTANT DATA ######################
    # Load environment variables from the .env file
    env_file_path = 'D:/Projects/.env'
    load_dotenv(env_file_path)
    # Giving output file name
    output_file_path = 'HOURLY.xlsx'
    # Load data from different sheets in 'promotion.xlsx' into DataFrames
    promotion_path = r'D:\Projects\promotion.xlsx'
    region_df = pd.read_excel(promotion_path, sheet_name='Region')
    aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
    paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
    types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

    ##################### ACCESS ENV VARIABLES ######################
    db_server = os.getenv("DB_SERVER")
    db_database = os.getenv("DB_DATABASE_SERGELI")
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_port = os.getenv("DB_PORT")
    db_driver_name = os.getenv("DB_DRIVER_NAME")

    ##################### PROCEDURE NAME ######################
    procedure_name = os.getenv("HOURLY_SHORT")  # THIS IS HOURLY DATA GATHERING

    ##################### DATE - MARCH ######################
    CURRENT_MONTH = 3
    CURRENT_YEAR = 2024
    today_date = datetime.now().strftime('%Y%m%d')
    tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')

    ##################### CONNECTION STRING AND SQL QUERY ######################
    # Construct the connection string
    conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
    engine = create_engine(conn_str)

    sql_query = f"""
        DECLARE @DateBegin DATE = ?;
        DECLARE @DateEnd DATE = ?;

        EXEC {procedure_name}
            @DateBegin = @DateBegin,
            @DateEnd = @DateBegin;
    """

    #####################  EXECUTION  ######################
    df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))

    ##################### BASIC FILTER ######################
    df['DataEntered'] = pd.to_datetime(df['DataEntered'])
    df = df[(df['DataEntered'].dt.month == CURRENT_MONTH) & (df['DataEntered'].dt.year == CURRENT_YEAR) & (
            df['DataEntered'].dt.date == datetime.now().date()) & df['Dname'].isin(
        ['Оптовая реализация', 'Финансовая скидка'])]

    df['B.Summ'] = df['BasePrice'] * df['Kolich']
    df['TotalAmount'] = df['oPrice'] * df['Kolich']

    region_df['ClientMan'] = region_df['ClientMan'].str.title()
    df['ClientMan'] = df['ClientMan'].str.title()

    df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
    df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')
    df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientMan', right_on='ClientMan', how='left')

    df['OXVAT'] = df['inn'].map(df['inn'].value_counts())

    df['inn_temp'] = pd.to_numeric(df['inn'], errors='coerce')
    types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
    df = pd.merge(df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp', how='left')

    ##################### ROZ | SET | OPT  ######################
    df['TYPE'].fillna('ROZ', inplace=True)
    df.loc[df['TYPE'] == 'ROZ', 'RegionType'] = df['Region']
    df.drop(['INN_temp', 'inn_temp'], axis=1, inplace=True)
    df.columns = ['Office', 'IncData', 'Number', 'Dname', 'ExpData', 'SerialNo', 'Number', 'Month', 'Year', 'Data',
                  'Goodid', 'Good', 'Producer', 'Client', 'Vid', 'type', 'inn', 'City', 'SalerMan', 'ClientMan',
                  'Store', 'StoreDep', 'DownPayment', 'PaymentTerm', 'BasePrice', 'Price', 'oPrice', 'Kolich', 'MarkUp',
                  'rMarkUp', 'p25', 'p100', 'OrderID', 'DataEntered', 'gkm_group', 'B.Summ', 'TotalAmount', 'Aksiya',
                  'Paket', 'Region', 'OXVAT', 'TYPE', 'RegionType']

    ##################### CONVERT TO CATEGORICAL DATA TYPE ######################
    categorical_columns = ['TYPE', 'Region', 'Aksiya', 'Paket', 'ClientMan', 'Dname', 'Vid', 'City', 'type', 'Store',
                           'StoreDep', 'RegionType', 'Month', 'Year', 'Office', 'Producer']
    df[categorical_columns] = df[categorical_columns].astype('category')
    df.sort_values(by='DataEntered', ascending=False, inplace=True)
    df.to_excel(output_file_path, index=False)

    ##################### FORMAT THE TABLE  ######################
    formatter(df, output_file_path)
    ##################### MODIFIED TIME OF THE FILE ######################

    if os.path.exists(output_file_path):
        # Get the size of the file in bytes
        file_size_bytes = os.path.getsize(output_file_path)

        # Convert bytes to kilobytes, megabytes, or gigabytes for readability
        file_size_kb = file_size_bytes / 1024.0
        file_size_mb = file_size_kb / 1024.0
        file_size_gb = file_size_mb / 1024.0

        print(
            f"File Size: {file_size_bytes} bytes, {file_size_kb:.2f} KB, {file_size_mb:.2f} MB, {file_size_gb:.2f} GB")

        # Get the last modification time in seconds since the epoch
        modification_time_seconds = os.path.getmtime(output_file_path)

        # Convert seconds since the epoch to a datetime object
        modification_time = datetime.fromtimestamp(modification_time_seconds)

        # Get the change time (metadata change time) in seconds since the epoch
        change_time_seconds = os.path.getctime(output_file_path)

        # Convert seconds since the epoch to a datetime object
        change_time = datetime.fromtimestamp(change_time_seconds)

        print(f"Last Modified Time: {modification_time.strftime('%H:%M')}")
        print(f"Change Time: {change_time.strftime('%H:%M')}")

    else:
        print("File not found.")


In [ ]:
hourly_generator('bekzod_hi', '6541236')

In [ ]:

##################### LOADING IMPORTANT DATA ######################
# Load environment variables from the .env file
env_file_path = 'D:/Projects/.env'
load_dotenv(env_file_path)
# Giving output file name
output_file_path = 'HOURLY.xlsx'
# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = r'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

##################### ACCESS ENV VARIABLES ######################
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_SERGELI")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

##################### PROCEDURE NAME ######################
procedure_name = os.getenv("HOURLY_SHORT")  # THIS IS HOURLY DATA GATHERING

##################### DATE - MARCH ######################
CURRENT_MONTH = 3
CURRENT_YEAR = 2024
today_date = datetime.now().strftime('%Y%m%d')
tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')

##################### CONNECTION STRING AND SQL QUERY ######################
# Construct the connection string
conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)

sql_query = f"""
    DECLARE @DateBegin DATE = ?;
    DECLARE @DateEnd DATE = ?;

    EXEC {procedure_name}
        @DateBegin = @DateBegin,
        @DateEnd = @DateBegin;
"""

#####################  EXECUTION  ######################
df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))

##################### BASIC FILTER ######################
df['DataEntered'] = pd.to_datetime(df['DataEntered'])
df.columns = ['DocKind', 'InvoiceNumber', 'GoodId', 'GoodName', 'Manufacturer', 'INN', 'ClientName', 'InvoiceManager',
              'ClientManager', 'PaymentTerm',
              'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered', 'BaseAmount', 'TotalAmount']

df = df[(df['DataEntered'].dt.month == CURRENT_MONTH) & (df['DataEntered'].dt.year == CURRENT_YEAR) & (
        df['DataEntered'].dt.date == datetime.now().date()) & df['DocKind'].isin(
    ['Оптовая реализация', 'Финансовая скидка'])]

In [ ]:
df.columns = ['DocKind', 'InvoiceNumber', 'GoodId', 'GoodName', 'Manufacturer', 'INN', 'ClientName', 'InvoiceManager',
              'ClientManager', 'PaymentTerm',
              'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered', 'BaseAmount', 'TotalAmount']

In [ ]:



def hourly_generator(login, password):
    ##################### LOADING IMPORTANT DATA ######################
    # Load environment variables from the .env file
    env_file_path = 'D:/Projects/.env'
    load_dotenv(env_file_path)
    # Giving output file name
    output_file_path = 'HOURLY.xlsx'
    # Load data from different sheets in 'promotion.xlsx' into DataFrames
    promotion_path = r'D:\Projects\promotion.xlsx'
    region_df = pd.read_excel(promotion_path, sheet_name='Region')
    aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
    paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
    types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

    ##################### ACCESS ENV VARIABLES ######################
    db_server = os.getenv("DB_SERVER")
    db_database = os.getenv("DB_DATABASE_SERGELI")
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_port = os.getenv("DB_PORT")
    db_driver_name = os.getenv("DB_DRIVER_NAME")

    ##################### PROCEDURE NAME ######################
    procedure_name = os.getenv("HOURLY_SHORT")  # THIS IS HOURLY DATA GATHERING

    ##################### DATE - MARCH ######################
    CURRENT_MONTH = 3
    CURRENT_YEAR = 2024
    today_date = datetime.now().strftime('%Y%m%d')
    tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')

    ##################### CONNECTION STRING AND SQL QUERY ######################
    # Construct the connection string
    conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
    engine = create_engine(conn_str)

    sql_query = f"""
        DECLARE @DateBegin DATE = ?;
        DECLARE @DateEnd DATE = ?;

        EXEC {procedure_name}
            @DateBegin = @DateBegin,
            @DateEnd = @DateBegin;
    """

    #####################  EXECUTION  ######################
    df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))

    ##################### BASIC FILTER ######################
    df['DataEntered'] = pd.to_datetime(df['DataEntered'])
    df.columns = ['DocKind', 'InvoiceNumber', 'Goodid', 'GoodName', 'Manufacturer', 'INN', 'ClientName',
                  'InvoiceManager',
                  'ClientMan', 'PaymentTerm',
                  'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered', 'BaseAmount', 'TotalAmount']

    df = df[(df['DataEntered'].dt.month == CURRENT_MONTH) & (df['DataEntered'].dt.year == CURRENT_YEAR) & (
            df['DataEntered'].dt.date == datetime.now().date()) & df['DocKind'].isin(
        ['Оптовая реализация', 'Финансовая скидка'])]

    region_df['ClientMan'] = region_df['ClientMan'].str.title()
    df['ClientMan'] = df['ClientMan'].str.title()

    df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
    df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')
    df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientMan', right_on='ClientMan', how='left')

    df['OXVAT'] = df['INN'].map(df['INN'].value_counts())

    df['inn_temp'] = pd.to_numeric(df['INN'], errors='coerce')
    types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
    df = pd.merge(df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp', how='left')

    ##################### ROZ | SET | OPT  ######################
    df['TYPE'].fillna('ROZ', inplace=True)
    df.loc[df['TYPE'] == 'ROZ', 'RegionType'] = df['Region']
    df.drop(['INN_temp', 'inn_temp'], axis=1, inplace=True)

    ##################### CONVERT TO CATEGORICAL DATA TYPE ######################
    df.sort_values(by='DataEntered', ascending=False, inplace=True)
    df.to_excel(output_file_path, index=False)

    ##################### FORMAT THE TABLE  ######################
    formatter(df, output_file_path)
    ##################### MODIFIED TIME OF THE FILE ######################

    if os.path.exists(output_file_path):
        # Get the size of the file in bytes
        file_size_bytes = os.path.getsize(output_file_path)

        # Convert bytes to kilobytes, megabytes, or gigabytes for readability
        file_size_kb = file_size_bytes / 1024.0
        file_size_mb = file_size_kb / 1024.0
        file_size_gb = file_size_mb / 1024.0

        print(
            f"File Size: {file_size_bytes} bytes, {file_size_kb:.2f} KB, {file_size_mb:.2f} MB, {file_size_gb:.2f} GB")

        # Get the last modification time in seconds since the epoch
        modification_time_seconds = os.path.getmtime(output_file_path)

        # Convert seconds since the epoch to a datetime object
        modification_time = datetime.fromtimestamp(modification_time_seconds)

        # Get the change time (metadata change time) in seconds since the epoch
        change_time_seconds = os.path.getctime(output_file_path)

        # Convert seconds since the epoch to a datetime object
        change_time = datetime.fromtimestamp(change_time_seconds)

        print(f"Last Modified Time: {modification_time.strftime('%H:%M')}")
        print(f"Change Time: {change_time.strftime('%H:%M')}")

    else:
        print("File not found.")


In [ ]:
hourly_generator('bekzod_hi', '6541236')

In [ ]:
import os
from datetime import datetime, timedelta  # For working with dates

import pandas as pd  # For working with DataFrames
from dotenv import load_dotenv
from sqlalchemy import create_engine  # For creating a connection engine

from reports.formatter import formatter


def hourly_generator():
    ##################### LOADING IMPORTANT DATA ######################
    # Load environment variables from the .env file
    env_file_path = 'D:/Projects/.env'
    load_dotenv(env_file_path)
    # Giving output file name
    output_file_path = 'HOURLY.xlsx'
    # Load data from different sheets in 'promotion.xlsx' into DataFrames
    promotion_path = r'D:\Projects\promotion.xlsx'
    region_df = pd.read_excel(promotion_path, sheet_name='Region')
    aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
    paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
    types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

    ##################### ACCESS ENV VARIABLES ######################
    db_server = os.getenv("DB_SERVER")
    db_database = os.getenv("DB_DATABASE_SERGELI")
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_port = os.getenv("DB_PORT")
    db_driver_name = os.getenv("DB_DRIVER_NAME")

    ##################### PROCEDURE NAME ######################
    procedure_name = os.getenv("HOURLY_SHORT")  # THIS IS HOURLY DATA GATHERING

    ##################### DATE - MARCH ######################
    CURRENT_MONTH = datetime.now().month
    CURRENT_YEAR = datetime.now().year
    today_date = datetime.now().strftime('%Y%m%d')
    tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')

    ##################### CONNECTION STRING AND SQL QUERY ######################
    # Construct the connection string
    conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
    engine = create_engine(conn_str)

    sql_query = f"""
        DECLARE @DateBegin DATE = ?;
        DECLARE @DateEnd DATE = ?;

        EXEC {procedure_name}
            @DateBegin = @DateBegin,
            @DateEnd = @DateBegin;
    """

    #####################  EXECUTION  ######################
    df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))

    ##################### BASIC FILTER ######################
    df['DataEntered'] = pd.to_datetime(df['DataEntered'])
    df.columns = ['DocKind', 'InvoiceNumber', 'Goodid', 'GoodName', 'Manufacturer', 'INN', 'ClientName',
                  'InvoiceManager',
                  'ClientMan', 'PaymentTerm',
                  'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered', 'BaseAmount', 'TotalAmount']

    df = df[(df['DataEntered'].dt.month == CURRENT_MONTH)
            & (df['DataEntered'].dt.year == CURRENT_YEAR)
            & (df['DataEntered'].dt.date == datetime.now().date())
            & df['DocKind'].isin(['Оптовая реализация', 'Финансовая скидка'])]
    # # 
    # region_df['ClientMan'] = region_df['ClientMan'].str.title()
    # df['ClientMan'] = df['ClientMan'].str.title()
    # 
    # df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
    # df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')
    # df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientMan', right_on='ClientMan', how='left')

    # df['OXVAT'] = df['INN'].map(df['INN'].value_counts())

    # df['inn_temp'] = pd.to_numeric(df['INN'], errors='coerce')
    # types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
    # df = pd.merge(df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp', how='left')

    ##################### ROZ | SET | OPT  ######################
    # df['TYPE'].fillna('ROZ', inplace=True)
    # df.loc[df['TYPE'] == 'ROZ', 'RegionType'] = df['Region']
    # df.drop(['INN_temp', 'inn_temp'], axis=1, inplace=True)

    ##################### CONVERT TO CATEGORICAL DATA TYPE ######################
    # df.sort_values(by='DataEntered', ascending=False, inplace=True)
    df.to_excel(output_file_path, index=False)

    ##################### FORMAT THE TABLE  ######################
    # formatter(df, output_file_path)
    ##################### MODIFIED TIME OF THE FILE ######################

    if os.path.exists(output_file_path):
        # Get the size of the file in bytes
        file_size_bytes = os.path.getsize(output_file_path)

        # Convert bytes to kilobytes, megabytes, or gigabytes for readability
        file_size_kb = file_size_bytes / 1024.0
        file_size_mb = file_size_kb / 1024.0
        file_size_gb = file_size_mb / 1024.0

        print(
            f"File Size: {file_size_bytes} bytes, {file_size_kb:.2f} KB, {file_size_mb:.2f} MB, {file_size_gb:.2f} GB")

        # Get the last modification time in seconds since the epoch
        modification_time_seconds = os.path.getmtime(output_file_path)

        # Convert seconds since the epoch to a datetime object
        modification_time = datetime.fromtimestamp(modification_time_seconds)

        # Get the change time (metadata change time) in seconds since the epoch
        change_time_seconds = os.path.getctime(output_file_path)

        # Convert seconds since the epoch to a datetime object
        change_time = datetime.fromtimestamp(change_time_seconds)

        print(f"Last Modified Time: {modification_time.strftime('%H:%M')}")
        print(f"Change Time: {change_time.strftime('%H:%M')}")

    else:
        print("File not found.")


In [ ]:
##################### LOADING IMPORTANT DATA ######################
# Load environment variables from the .env file
env_file_path = 'D:/Projects/.env'
load_dotenv(env_file_path)
# Giving output file name
output_file_path = 'HOURLY.xlsx'
# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = r'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

##################### ACCESS ENV VARIABLES ######################
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_SERGELI")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

##################### PROCEDURE NAME ######################
procedure_name = os.getenv("HOURLY_SHORT")  # THIS IS HOURLY DATA GATHERING

##################### DATE - MARCH ######################
CURRENT_MONTH = datetime.now().month
CURRENT_YEAR = datetime.now().year
today_date = datetime.now().strftime('%Y%m%d')
tomorrow_date = (datetime.now() + timedelta(days=1)).strftime('%Y%m%d')

##################### CONNECTION STRING AND SQL QUERY ######################
# Construct the connection string
conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)

sql_query = f"""
    DECLARE @DateBegin DATE = ?;
    DECLARE @DateEnd DATE = ?;

    EXEC {procedure_name}
        @DateBegin = @DateBegin,
        @DateEnd = @DateBegin;
"""

#####################  EXECUTION  ######################
df = pd.read_sql_query(sql_query, engine, params=(today_date, tomorrow_date))

##################### BASIC FILTER ######################
df.columns = ['DocKind', 'InvoiceNumber', 'Goodid', 'GoodName', 'Manufacturer', 'INN', 'ClientName',
              'InvoiceManager',
              'ClientMan', 'PaymentTerm',
              'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered', 'BaseAmount', 'TotalAmount']

df['DataEntered'] = pd.to_datetime(df['DataEntered'])

df = df[(df['DataEntered'].dt.month == CURRENT_MONTH)
        & (df['DataEntered'].dt.year == CURRENT_YEAR)
        & (df['DataEntered'].dt.date == today_date)
        & df['DocKind'].isin(['Оптовая реализация', 'Финансовая скидка'])]

region_df['ClientMan'] = region_df['ClientMan'].str.title()
df['ClientMan'] = df['ClientMan'].str.title()

df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientMan', right_on='ClientMan', how='left')

# df['OXVAT'] = df['INN'].map(df['INN'].value_counts())

# df['inn_temp'] = pd.to_numeric(df['INN'], errors='coerce')
# types_df['INN_temp'] = pd.to_numeric(types_df['INN'], errors='coerce')
# df = pd.merge(df, types_df[['INN_temp', 'TYPE', 'RegionType']], left_on='inn_temp', right_on='INN_temp', how='left')

##################### ROZ | SET | OPT  ######################
# df['TYPE'] = df['TYPE'].fillna('ROZ')
# df.loc[df['TYPE'] == 'ROZ', 'RegionType'] = df['Region']
# df.drop(['INN_temp', 'inn_temp'], axis=1, inplace=True)

##################### CONVERT TO CATEGORICAL DATA TYPE ######################
df.sort_values(by='DataEntered', ascending=False, inplace=True)
df.to_excel(output_file_path, index=False)

##################### FORMAT THE TABLE  ######################
formatter(df, output_file_path)
##################### MODIFIED TIME OF THE FILE ######################

In [ ]:
df

In [ ]:



def create_excel(names):
    wb = openpyxl.Workbook()
    sheet = wb.active

    # Writing names to the first row
    for col, name in enumerate(names, start=1):
        cell = sheet.cell(row=1, column=col)
        cell.value = name
        cell.alignment = Alignment(horizontal='center')

    # Writing sub-columns for each name
    sub_columns = ['план', 'сумма', 'факт кол-во', 'факт сумма']
    for i, name in enumerate(names):
        start_col = i * len(sub_columns) + 1
        end_col = (i + 1) * len(sub_columns)
        merged_cell = sheet.cell(row=2, column=start_col)
        merged_cell.value = name
        sheet.merge_cells(start_row=2, start_column=start_col, end_row=2, end_column=end_col)
        for j, sub_col in enumerate(sub_columns, start=start_col):
            col_letter = get_column_letter(j)
            cell = sheet.cell(row=3, column=j)
            cell.value = sub_col
            cell.alignment = Alignment(horizontal='center')

    # Randomly colorize each name section
    for i, name in enumerate(names):
        start_col = i * len(sub_columns) + 1
        end_col = (i + 1) * len(sub_columns)
        color = "%06x" % randint(0, 0xFFFFFF)  # Generate a random color
        fill = PatternFill(start_color=color, end_color=color, fill_type="solid")
        for col in range(start_col, end_col + 1):
            for row in range(1, 4):
                cell = sheet.cell(row=row, column=col)
                cell.fill = fill

    # Save the workbook
    wb.save("output.xlsx")


names = [
    'АСТОР АЛЬЯНС ПОКУПАТЕЛЬ'
    , 'Аэро фарм груп покупатель'
    , 'Бек Барака Самарканд ИК ХК'
    , 'Бест фарм гранд НАМ'
    , 'Билолхон мед фарм сервис МЧЖ'
    , 'Биотек Фарм ООО'
    , 'Верона Фергана Покупатель'
    , 'ВИЗА ГОЛД'
    , 'Витрум мед лине'
    , 'Генисис фарма '
    , 'Генисис фарма реализация '
    , 'Гранд Фарм Трейд ООО'
    , 'Дел Фарм'
    , 'Доришунос мед фарм'
    , 'Журабек-Ота К.Т.К.К. нам'
    , 'Зента фарм покупатель '
    , 'Инсоф Барака Фарм '
    , 'Искандар Гранд Фарм'
    , 'Мадина-Фарм Тошкент'
    , 'Макку-Фарм '
    , 'Мемори фарм А Г'
    , 'мустанг групп дист'
    , 'Мухаммад-Солих нам'
    , 'Мухтасар Коканд'
    , 'Нафис Файз '
    , 'ПАРАДАЙС ФАРМ'
    , 'РЕФЛЕКС СТАРТ ФАРМ МЕДИКАЛ'
    , 'Саид Мед Фарм Сервис  НАМ'
    , 'Самина Фарм энд Бро ООО'
    , 'Самфарм'
    , 'Унф Инавцион Фарм Нам'
    , 'Фарм Асия Коканд'
    , 'Фарм люкс инвест покупатель'
    , 'Фарм Фемида продажа '
    , 'Фарма Чойс покупатель'
    , 'Фармакси'
    , 'Хасан Хусан Фарм МЧЖ'
    , 'Хуршида Ентер Делюкс',
    'ЮМАКСФАРМ СЕРВИС МЧЖ'
    , 'Янг Фарм'
    , 'Навкирон Фарм анд'
    , 'ПРОТЕК МЕД'
    , 'Толмомед МЧЖ'
    , 'Курацио'
    , '(Бух) Омега фарм  Бухара ооо'
    , 'Мирзохид Искандар ЧП ШАХРИСАБС'
    , '(Нук) Руди Фарм',
    '(нук) Туран импекс'
    , 'Регис Фарма Фер'

]

create_excel(names)


In [ ]:



def get_contrast_color(hex_color):
    # Convert hex color to RGB
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)

    # Calculate brightness
    brightness = (r * 299 + g * 587 + b * 114) / 1000

    # Return black or white based on brightness
    return '000000' if brightness > 128 else 'FFFFFF'


def create_excel(names):
    wb = openpyxl.Workbook()
    sheet = wb.active

    # Writing names to the first row
    for col, name in enumerate(names, start=1):
        cell = sheet.cell(row=1, column=col)
        cell.value = name
        cell.alignment = Alignment(horizontal='center')

    # Writing sub-columns for each name
    sub_columns = ['план', 'сумма', 'факт кол-во', 'факт сумма']
    for i, name in enumerate(names):
        start_col = i * len(sub_columns) + 1
        end_col = (i + 1) * len(sub_columns)
        merged_cell = sheet.cell(row=2, column=start_col)
        merged_cell.value = name
        sheet.merge_cells(start_row=2, start_column=start_col, end_row=2, end_column=end_col)
        for j, sub_col in enumerate(sub_columns, start=start_col):
            col_letter = get_column_letter(j)
            cell = sheet.cell(row=3, column=j)
            cell.value = sub_col
            cell.alignment = Alignment(horizontal='center')

    # Randomly colorize each name section
    for i, name in enumerate(names):
        start_col = i * len(sub_columns) + 1
        end_col = (i + 1) * len(sub_columns)
        color = "%06x" % randint(0, 0xFFFFFF)  # Generate a random color
        text_color = get_contrast_color(color)
        fill = PatternFill(start_color=color, end_color=color, fill_type="solid")
        font = Font(color=text_color)
        for col in range(start_col, end_col + 1):
            for row in range(1, 4):
                cell = sheet.cell(row=row, column=col)
                cell.fill = fill
                cell.font = font

    # Save the workbook
    wb.save("output.xlsx")


names = [
    'АСТОР АЛЬЯНС ПОКУПАТЕЛЬ'
    , 'Аэро фарм груп покупатель'
    , 'Бек Барака Самарканд ИК ХК'
    , 'Бест фарм гранд НАМ'
    , 'Билолхон мед фарм сервис МЧЖ'
    , 'Биотек Фарм ООО'
    , 'Верона Фергана Покупатель'
    , 'ВИЗА ГОЛД'
    , 'Витрум мед лине'
    , 'Генисис фарма '
    , 'Генисис фарма реализация '
    , 'Гранд Фарм Трейд ООО'
    , 'Дел Фарм'
    , 'Доришунос мед фарм'
    , 'Журабек-Ота К.Т.К.К. нам'
    , 'Зента фарм покупатель '
    , 'Инсоф Барака Фарм '
    , 'Искандар Гранд Фарм'
    , 'Мадина-Фарм Тошкент'
    , 'Макку-Фарм '
    , 'Мемори фарм А Г'
    , 'мустанг групп дист'
    , 'Мухаммад-Солих нам'
    , 'Мухтасар Коканд'
    , 'Нафис Файз '
    , 'ПАРАДАЙС ФАРМ'
    , 'РЕФЛЕКС СТАРТ ФАРМ МЕДИКАЛ'
    , 'Саид Мед Фарм Сервис  НАМ'
    , 'Самина Фарм энд Бро ООО'
    , 'Самфарм'
    , 'Унф Инавцион Фарм Нам'
    , 'Фарм Асия Коканд'
    , 'Фарм люкс инвест покупатель'
    , 'Фарм Фемида продажа '
    , 'Фарма Чойс покупатель'
    , 'Фармакси'
    , 'Хасан Хусан Фарм МЧЖ'
    , 'Хуршида Ентер Делюкс',
    'ЮМАКСФАРМ СЕРВИС МЧЖ'
    , 'Янг Фарм'
    , 'Навкирон Фарм анд'
    , 'ПРОТЕК МЕД'
    , 'Толмомед МЧЖ'
    , 'Курацио'
    , '(Бух) Омега фарм  Бухара ооо'
    , 'Мирзохид Искандар ЧП ШАХРИСАБС'
    , '(Нук) Руди Фарм',
    '(нук) Туран импекс'
    , 'Регис Фарма Фер'

]

create_excel(names)


In [ ]:
import openpyxl
from openpyxl.styles import Alignment, PatternFill, Font
from openpyxl.utils import get_column_letter
from random import randint


def get_contrast_color(hex_color):
    # Convert hex color to RGB
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)

    # Calculate brightness
    brightness = (r * 299 + g * 587 + b * 114) / 1000

    # Return black or white based on brightness
    return '000000' if brightness > 128 else 'FFFFFF'


def create_excel(names):
    wb = openpyxl.Workbook()
    sheet = wb.active

    # Writing names to the first row
    for col, name in enumerate(names, start=1):
        cell = sheet.cell(row=1, column=col)
        cell.value = name
        cell.alignment = Alignment(horizontal='center')

    # Writing sub-columns for each name
    sub_columns = ['план', 'сумма', 'факт кол-во', 'факт сумма']
    for i, name in enumerate(names):
        start_col = i * len(sub_columns) + 1
        end_col = (i + 1) * len(sub_columns)
        merged_cell = sheet.cell(row=2, column=start_col)
        merged_cell.value = name
        sheet.merge_cells(start_row=2, start_column=start_col, end_row=2, end_column=end_col)
        for j, sub_col in enumerate(sub_columns, start=start_col):
            col_letter = get_column_letter(j)
            cell = sheet.cell(row=3, column=j)
            cell.value = sub_col
            cell.alignment = Alignment(horizontal='center')

    # Randomly colorize each name section
    for i, name in enumerate(names):
        start_col = i * len(sub_columns) + 1
        end_col = (i + 1) * len(sub_columns)
        color = "%06x" % randint(0, 0xFFFFFF)  # Generate a random color
        text_color = get_contrast_color(color)
        fill = PatternFill(start_color=color, end_color=color, fill_type="solid")
        font = Font(color=text_color)
        for col in range(start_col, end_col + 1):
            for row in range(1, sheet.max_row + 1):
                cell = sheet.cell(row=row, column=col)
                cell.fill = fill
                cell.font = font

    # Save the workbook
    wb.save("output.xlsx")


names = [
    'АСТОР АЛЬЯНС ПОКУПАТЕЛЬ'
    , 'Аэро фарм груп покупатель'
    , 'Бек Барака Самарканд ИК ХК'
    , 'Бест фарм гранд НАМ'
    , 'Билолхон мед фарм сервис МЧЖ'
    , 'Биотек Фарм ООО'
    , 'Верона Фергана Покупатель'
    , 'ВИЗА ГОЛД'
    , 'Витрум мед лине'
    , 'Генисис фарма '
    , 'Генисис фарма реализация '
    , 'Гранд Фарм Трейд ООО'
    , 'Дел Фарм'
    , 'Доришунос мед фарм'
    , 'Журабек-Ота К.Т.К.К. нам'
    , 'Зента фарм покупатель '
    , 'Инсоф Барака Фарм '
    , 'Искандар Гранд Фарм'
    , 'Мадина-Фарм Тошкент'
    , 'Макку-Фарм '
    , 'Мемори фарм А Г'
    , 'мустанг групп дист'
    , 'Мухаммад-Солих нам'
    , 'Мухтасар Коканд'
    , 'Нафис Файз '
    , 'ПАРАДАЙС ФАРМ'
    , 'РЕФЛЕКС СТАРТ ФАРМ МЕДИКАЛ'
    , 'Саид Мед Фарм Сервис  НАМ'
    , 'Самина Фарм энд Бро ООО'
    , 'Самфарм'
    , 'Унф Инавцион Фарм Нам'
    , 'Фарм Асия Коканд'
    , 'Фарм люкс инвест покупатель'
    , 'Фарм Фемида продажа '
    , 'Фарма Чойс покупатель'
    , 'Фармакси'
    , 'Хасан Хусан Фарм МЧЖ'
    , 'Хуршида Ентер Делюкс',
    'ЮМАКСФАРМ СЕРВИС МЧЖ'
    , 'Янг Фарм'
    , 'Навкирон Фарм анд'
    , 'ПРОТЕК МЕД'
    , 'Толмомед МЧЖ'
    , 'Курацио'
    , '(Бух) Омега фарм  Бухара ооо'
    , 'Мирзохид Искандар ЧП ШАХРИСАБС'
    , '(Нук) Руди Фарм',
    '(нук) Туран импекс'
    , 'Регис Фарма Фер'

]
create_excel(names)

In [ ]:
import os


def delete_corrupted_folder(folder_path):
    try:
        # Try to delete the folder using os.system
        os.system(f'rmdir /s /q "{folder_path}"')
        print("Folder deleted successfully.")
    except Exception as e:
        print(f"Error: {e}")
        print("Unable to delete the folder.")


folder_path = r'F:\1'  # Path to the folder you want to delete
delete_corrupted_folder(folder_path)


In [ ]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the website
url = 'https://bank.uz/uz/currency'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')
# Extract "Sotish" (sell) data
sell_data = soup.find('div', class_='bc-inner-blocks-right').find_all('div', class_='bc-inner-block-left-texts')

# Extract "Sotib olish" (buy) data
buy_data = soup.find('div', class_='bc-inner-block-left').find_all('div', class_='bc-inner-block-left-texts')

# Print the extracted data
print(f"{'Sotish (Sell)':<30}{'Sotib olish (Buy)'}")
print(f"{'Bank Name':<25}{'Rate':<15}{'Bank Name':<25}{'Rate'}")
for sell, buy in zip(sell_data, buy_data):
    sell_bank_name = sell.find('span', class_='medium-text').text.strip()
    sell_rate = sell.find('span', class_='medium-text green-date').text.strip()
    buy_bank_name = buy.find('span', class_='medium-text').text.strip()
    buy_rate = buy.find('span', class_='medium-text green-date').text.strip()
    print(f"{sell_bank_name:<25}{sell_rate:<15}{buy_bank_name:<25}{buy_rate}")


In [ ]:
import locale
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from io import BytesIO
import datetime


# Function to get today's date in a formatted string
def get_today_date():
    locale.setlocale(locale.LC_TIME, 'uz_UZ.UTF-8')

    today = datetime.date.today()
    return today.strftime("%d - %B, %Y-yil")


# Send a GET request to the website
url = 'https://bank.uz/uz/currency'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract "Sotish" (sell) data
sell_data = soup.find('div', class_='bc-inner-blocks-right').find_all('div', class_='bc-inner-block-left-texts')

# Extract "Sotib olish" (buy) data
buy_data = soup.find('div', class_='bc-inner-block-left').find_all('div', class_='bc-inner-block-left-texts')

# Store data in lists
sell_bank_names = [sell.find('span', class_='medium-text').text.strip() for sell in sell_data]
sell_rates = [sell.find('span', class_='medium-text green-date').text.strip() for sell in sell_data]
buy_bank_names = [buy.find('span', class_='medium-text').text.strip() for buy in buy_data]
buy_rates = [buy.find('span', class_='medium-text green-date').text.strip() for buy in buy_data]


# Plot the exchange rates data as a table and send it as a photo
def plot_exchange_rates():
    # Create figure and plot the table
    fig, ax = plt.subplots(figsize=(20, 12))
    ax.axis('off')
    ax.axis('tight')

    # Add title with today's date
    ax.set_title(f"Dollar Kursi: {get_today_date()}",
                 fontsize=12,
                 fontweight='bold',
                 color='blue',
                 loc='left')

    # Define table data with headers and content
    table_data = [['Bank Nomi', 'Sotish (Sell)', 'Bank Nomi', 'Sotib olish (Buy)']]
    for sell_bank, sell_rate, buy_bank, buy_rate in zip(sell_bank_names, sell_rates, buy_bank_names, buy_rates):
        table_data.append([sell_bank, sell_rate, buy_bank, buy_rate])

    # Plot the table with styling
    ax.table(cellText=table_data,
             colLabels=None, cellLoc='center',
             loc='center', colColours=['lightgrey'] * 4,
             cellColours=[['white', 'lightcyan'] * 2] * len(table_data),
             colWidths=[0.25, 0.15, 0.25, 0.15]
             )

    # Adjust padding and margin between rows and columns
    cell_dict = \
        ax.table(cellText=table_data, colLabels=None, cellLoc='center', loc='center', colColours=['lightgrey'] * 4,
                 cellColours=[['white', 'lightcyan'] * 2] * len(table_data),
                 # colWidths=[0.25, 0.15, 0.25, 0.15]
                 ).properties()['children']

    for cell in cell_dict:
        cell._text.set_fontsize(25)
        cell.set_linewidth(2)
        cell.set_height(.045)
        cell.set_edgecolor('black')

    # Save the plot as an image
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)


# Call the function to plot and send exchange rates as a table photo
plot_exchange_rates()


In [ ]:
import locale
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from io import BytesIO
import datetime

# Function to get today's date in a formatted string
def get_today_date():
    locale.setlocale(locale.LC_TIME, 'uz_UZ.UTF-8')
    today = datetime.date.today()
    return today.strftime("%d - %B, %Y-yil")

# Send a GET request to the website
url = 'https://bank.uz/uz/currency'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract "Sotish" (sell) data
sell_data = soup.find('div', class_='bc-inner-blocks-right').find_all('div', class_='bc-inner-block-left-texts')

# Extract "Sotib olish" (buy) data
buy_data = soup.find('div', class_='bc-inner-block-left').find_all('div', class_='bc-inner-block-left-texts')

# Store data in lists
sell_bank_names = [sell.find('span', class_='medium-text').text.strip() for sell in sell_data]
sell_rates = [sell.find('span', class_='medium-text green-date').text.strip() for sell in sell_data]
buy_bank_names = [buy.find('span', class_='medium-text').text.strip() for buy in buy_data]
buy_rates = [buy.find('span', class_='medium-text green-date').text.strip() for buy in buy_data]

# Define headers
headers = [['Bank Nomi', 'Sotish (Sell)', 'Bank Nomi', 'Sotib olish (Buy)']]
# Plot the exchange rates data as a table and send it as a photo
def plot_exchange_rates():
    # Create figure and plot the table
    fig, ax = plt.subplots(figsize=(12,8))
    ax.axis('off')
    # ax.axis('tight')
    # Define table data with headers and content
    table_data = [headers[0]]
    for sell_bank, sell_rate, buy_bank, buy_rate in zip(sell_bank_names, sell_rates, buy_bank_names, buy_rates):
        table_data.append([sell_bank, sell_rate, buy_bank, buy_rate])

    # Plot the table with styling
    ax.table(cellText=table_data, colLabels=None, cellLoc='center', loc='center', colColours=['lightgrey']*4, cellColours=[['lightgrey']*4]+[['white', 'lightcyan']*2]*len(table_data[1:]), 
             # colWidths=[0.25, 0.15, 0.25, 0.15]
             )

    # Adjust padding and margin between rows and columns
    cell_dict = ax.table(cellText=table_data, colLabels=None, cellLoc='center', loc='center', colColours=['lightgrey']*4, cellColours=[['lightgrey']*4]+[['white', 'lightcyan']*2]*len(table_data[1:]), 
                         # colWidths=[0.25, 0.15, 0.25, 0.15]
                         ).properties()['children']
    
    for cell in cell_dict:
        cell._text.set_fontsize(25)
        cell.set_linewidth(2)
        cell.set_height(.045)
        cell.set_edgecolor('black')

    # Save the plot as an image
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)

# Call the function to plot and send exchange rates as a table photo
plot_exchange_rates()


In [ ]:
import locale
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from io import BytesIO
import datetime

# Function to get today's date in a formatted string
def get_today_date():
    locale.setlocale(locale.LC_TIME, 'uz_UZ.UTF-8')
    today = datetime.date.today()
    return today.strftime("%d - %B, %Y-yil")

# Send a GET request to the website
url = 'https://bank.uz/uz/currency'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract "Sotish" (sell) data
sell_data = soup.find('div', class_='bc-inner-blocks-right').find_all('div', class_='bc-inner-block-left-texts')

# Extract "Sotib olish" (buy) data
buy_data = soup.find('div', class_='bc-inner-block-left').find_all('div', class_='bc-inner-block-left-texts')

# Store data in lists
sell_bank_names = [sell.find('span', class_='medium-text').text.strip() for sell in sell_data]
sell_rates = [sell.find('span', class_='medium-text green-date').text.strip() for sell in sell_data]
buy_bank_names = [buy.find('span', class_='medium-text').text.strip() for buy in buy_data]
buy_rates = [buy.find('span', class_='medium-text green-date').text.strip() for buy in buy_data]

# Define headers
headers = [['Bank Nomi', 'Sotish (Sell)', 'Bank Nomi', 'Sotib olish (Buy)']]

# Plot the exchange rates data as a table and send it as a photo
def plot_exchange_rates():
    # Create figure and plot the table
    fig, ax = plt.subplots(figsize=(20, 12))
    ax.axis('off')
    ax.axis('tight')

    # Define table data with headers and content
    table_data = [headers[0]]  # Only headers without empty row before
    for sell_bank, sell_rate, buy_bank, buy_rate in zip(sell_bank_names, sell_rates, buy_bank_names, buy_rates):
        table_data.append([sell_bank, sell_rate, buy_bank, buy_rate])

    # Plot the table with styling
    ax.table(cellText=table_data, 
             colLabels=None, cellLoc='center', 
             loc='center', 
             colColours=['lightgrey']*4, 
             cellColours=[['lightgrey']*4]+[['#C0C0C0']*4]+[['white', 'lightcyan']*2]*len(table_data[2:]), 
             colWidths=[0.25, 0.15, 0.25, 0.15]
             )

    # Adjust padding and margin between rows and columns
    cell_dict = ax.table(cellText=table_data, colLabels=None, cellLoc='center', loc='center', colColours=['lightgrey']*4, cellColours=[['#B5C18E']*4]+[['white', 'lightcyan']*2]*len(table_data[2:]), 
                         colWidths=[0.25, 0.15, 0.25, 0.15]
                         ).properties()['children']
    
    for cell in cell_dict:
        cell._text.set_fontsize(25)
        cell.set_linewidth(2)
        cell.set_height(.05)
        cell.set_edgecolor('black')

    # Save the plot as an image
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)

# Call the function to plot and send exchange rates as a table photo
plot_exchange_rates()
